# Tamizh Data Cleaning

This Code is extract useful information form Tamizh data-set from (Wikimedia). 

Informations Planned to extract:
  
  1. Nouns
  2. multiple word - similar meaning
  3. Categories
  
  
 Data leaning part
  1. Extract only tamil sentence

Import relevant libraries

In [1]:
import pandas as pd
import re
import regex

Load the data

In [2]:
data = pd.read_csv("../data/tawiki_pages_articles.csv")

Define variable to save all the extracted information

1. Wikipedia article clean texts
2. Nouns (here nouns are considered as value bounded in box)
3. Curl Values
4. Heading values for articles
5. Tamil - English combined
6. Categories

In [3]:
#This will be considered as global variables #hope we have enought RAM for this slliness

boxed_values = []
curlBoxed_values = []
heading_values = []
boxed_and_curls = []
categories = []

Define regular expressoin syntax

And extract information functions



In [4]:
re_boxed_values = "\[[^\]^\[]*\]"
re_curlBoxed_values = "\{[^\{]*\}"
re_heading_values = "\=\=([^\=]*)\=\="
re_boxed_and_curls = "\[[^\]^\[]*\]\] \([^\(^\)]*\)"
re_categories = "\[\[பகுப்பு:([^\]]*)\]"

def parse_data(text,title):
  boxed_values.append({'title':title,'box':list(set(re.findall(re_boxed_values,text,re.M)))})
  curlBoxed_values.append({'title':title,'curl':list(set(re.findall(re_curlBoxed_values,text,re.M)))})
  heading_values.append({'title':title,'heading':list(set(re.findall(re_heading_values,text,re.M)))})
  boxed_and_curls.append({'title':title,'boxcurl':list(set(re.findall(re_boxed_and_curls,text,re.M)))})
  categories.append({'title':title,'categories':list(set(re.findall(re_categories,text,re.M)))})

Cleaning is required with multiple string needs to be eliminated with regex

Here I used "regex" function to extract tamil chars from the string (Note : not re)

In [5]:
#set of regex rules to extract patterns

re_braces = "\([^\)]*\)"
re_curl_braces ="\{\{[^\}\{]*\}\}"
re_single_curl = "\{[^\}\{]*\}"
# re_curl_braces ="\{[^\}]*\}"
# re_double_curl_braces = "\{[^\{]*\{[^\}]*\}\}[^\}]*\}\}"
re_category = "\[\[பகுப்பு:[^\]]*\]"
re_header = "(\=\=[^\=]*\=\=)"
re_img = "\[\[படிமம்:.*\]\]"
re_ref1 = "<ref.*?/>"
re_ref2 = "<ref.*?</ref>"

def clean_data(text):
  text = re.sub(re_braces,"",text)
  text = re.sub(re_ref2,"",text)
  text = re.sub(re_ref1,"",text)
  text = re.sub(re_curl_braces,"",text)
  text = re.sub(re_curl_braces,"",text)
  text = re.sub(re_curl_braces,"",text)
  text = re.sub(re_single_curl,"",text)
  text = re.sub(re_category,"",text)
  text = re.sub(re_header,"",text)
  text ="".join(regex.findall("[\p{Tamil} ]",text))
  text = re.sub("\s\s+" , " ", text)
  return text

Even some entire sections in the article is not needed for the language modelling purpose. 

We can eliminate those from the existance :-p 

In [6]:
section_rm= ['மேற்கோள்கள்','இவற்றையும் பார்க்கவும்','வெளி இணைப்புகள்','நூற்பட்டியல்','துணை நூல்கள்']

def remove_sections(text):
  text = re.sub(re_img,"",text)
  article = ""
  for para in text.split("\n\n"):
    if not any([section in para for section in section_rm]):
      article += para 
      
  return article

In [7]:
#remove_sections(data.text[8])

So Here we go !!!!!!!!!!!!

We have all the funtions for 

1. Extract the relevant information from the article
2. Clean the data
3. Extracting the pure -(information) from the article 


In [8]:
#combine all these into a single funciton

def extract_tawiki(text,title):
  if type(text)==str:
    text = remove_sections(text)
    parse_data(text,title)
    clean_text = clean_data(text)
    return clean_text
  else:
    return ""
  

Apply the same logic to all data in the dataframe 



In [9]:
#text data
tawiki_data= []


#Other useful information
boxed_values = []
curlBoxed_values = []
heading_values = []
boxed_and_curls = []
categories = []


for i in range(data.shape[0]):
  tawiki_data.append(extract_tawiki(data.text.iloc[i],data.title.iloc[i]))

In [10]:
#data to be saved
tawiki_data_df = pd.DataFrame(tawiki_data,columns=['Content'])
boxed_values_df = pd.DataFrame(boxed_values)
#curl_boxed_values = pd.DataFrame(curlBoxed_values)
boxed_and_curls_df = pd.DataFrame(boxed_and_curls)
categories_df = pd.DataFrame(categories)

In [11]:
categories_df.head()

,title,categories
0,முதற் பக்கம்,[]
1,விக்கிப்பீடியா:கலந்துரையாடல்,[விக்கிப்பீடியா ஆலமரத்தடி]
2,விக்கிப்பீடியா:புதுப் பயனர் பக்கம்,[விக்கிப்பீடியா உதவி]
3,விக்கிப்பீடியா:விக்கிப்பீடியர்கள்,[விக்கிப்பீடியர்கள்]
4,கட்டிடக்கலை,[கட்டிடக்கலை| ]


Save all the results to disk.



In [12]:
tawiki_data_df.to_csv("../data/tawiki_data_df.csv",index=False)
boxed_values_df.to_csv("../data/boxed_value_df.csv",index =False)
boxed_and_curls_df.to_csv("../data/boxednCurls_df.csv",index=False)
categories_df.to_csv("../data/categories_df.csv",index=False)